## Opdracht: Build your own neural net from scratch.
### Auteur: Aghead Bilal
### Datum: 2023-01-03
### Versie: 4.0

In [1]:
# Importeren van de benodigde libraries
import numpy as np

Het doel is het implementeren van een complete neuraal net met alleen dense layers dat te trainen is voor een bepaalde taak.
De taak die we gaan gebruiken is Maak en test netwerk en test hem b.v op de xor functie. 
Stappenplan.

- 1: Schrijf de activatie functies als Relu en Sigmoid dusdanig dat als aangeroepen met een numpy array ze een array van dezelfde vorm teruggeven waarin de functie op elk element is toegepast. Maak ook de afgeleide fucnties. Implementeer ook de Softmax functie voor de outputlaag. <br>

- 2: Verzin een datastructuur (b.v python list) waarin je een laag van een NN kan opslaan, Denk aan minimaal weights, biases en activatie functies. Maak ook de functies om een laag te initialiseren. <br>

- 3: Schrijf de feedforward functie die de input neemt en een geheel net doorrekend. <br>

- 4: Bepaal de output error bij een gegeven in- en output en back-propagate door het netwerk zodat je van iedere laag de error kent.  <br>

- 5: Update de weights en biases gebaseerd op de errors in de laag en de afgeleide van de gebruikte errorfunctie en activatiefunctie. <br>

- 6: Schrijf een functie die een netwerk kan trainen op een gegeven dataset. <br>

- 7: Schrijf een functie die een netwerk kan testen op een gegeven dataset. <br>

- 8: Maak een netwerk dat de XOR functie kan leren. <br>

### 1: Schrijf de activatie functies als Relu en Sigmoid dusdanig dat als aangeroepen met een numpy array ze een array van dezelfde vorm teruggeven waarin de functie op elk element is toegepast. Maak ook de afgeleide fucnties. Implementeer ook de Softmax functie voor de outputlaag.

In [2]:
# Relu functie
def relu(x):
    return np.maximum(0, x)

# Relu afgeleide functie
def relu_derivative(x):
    x[x<=0] = 0
    x[x>0] = 1
    return x

# Sigmoid functie
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Sigmoid afgeleide functie
def sigmoid_derivative(x):
    return x * (1 - x)

# Softmax functie
def softmax(x):
    exps = np.exp(x - np.max(x))
    return exps / np.sum(exps)

### 2: Verzin een datastructuur (b.v python list) waarin je een laag van een NN kan opslaan, Denk aan minimaal weights, biases en activatie functies. Maak ook de functies om een laag te initialiseren.

In [3]:
# Datastructuur
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons, activation=None, weights=None, biases=None):
        self.weights = weights if weights is not None else 0.1 * np.random.randn(n_inputs, n_neurons)
        self.biases = biases if biases is not None else np.zeros((1, n_neurons))
        self.activation = activation
        self.output = np.zeros((1, n_neurons))

    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

        if self.activation is not None:
            self.output = self.activation(self.output)

### 3: Schrijf de feedforward functie die de input neemt en een geheel net doorrekend.

In [4]:
# Feedforward functie
def feedforward(X, network):
    layer_outputs = []
    for layer in network:
        layer.forward(X)
        layer_outputs.append(layer.output)
        X = layer.output
    return layer_outputs

### 4: Bepaal de output error bij een gegeven in- en output en back-propagate door het netwerk zodat je van iedere laag de error kent

In [5]:
# Backpropagation functie
def backpropagation(y, network, layer_outputs):
    errors = []
    for i in reversed(range(len(network))):
        layer = network[i]
        if layer == network[-1]:
            errors.insert(0, y - layer_outputs[i])
        else:
            errors.insert(0, np.dot(errors[0], network[i + 1].weights.T))
        layer.error = errors[0] * relu_derivative(layer_outputs[i])

### 5: Update de weights en biases gebaseerd op de errors in de laag en de afgeleide van de gebruikte errorfunctie en activatiefunctie.

In [6]:
# Update functie
def update(network, X, learning_rate):
    for i in range(len(network)):
        layer = network[i]
        if i == 0:
            inputs = X
        else:
            inputs = network[i - 1].output
        layer.weights += learning_rate * np.dot(inputs.T, layer.error)
        layer.biases += learning_rate * np.sum(layer.error, axis=0, keepdims=True)

### 6: Schrijf een functie die een netwerk kan trainen op een gegeven dataset.

In [7]:
# Train functie
def train(X, y, network, learning_rate, n_iterations):
    for i in range(n_iterations):
        layer_outputs = feedforward(X, network)
        backpropagation(y, network, layer_outputs)
        update(network, X, learning_rate)

### 7: Schrijf een functie die een netwerk kan testen op een gegeven dataset.

In [8]:
# Test functie
def test(X, y, network):
    layer_outputs = feedforward(X, network)
    predictions = np.argmax(layer_outputs[-1], axis=1)
    accuracy = np.mean(predictions == y)
    return accuracy

### 8: Maak een netwerk dat de XOR functie kan leren.

In [21]:
# XOR data
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])

# Maak netwerk
network = []
network.append(Layer_Dense(2, 4, activation=relu))
network.append(Layer_Dense(4, 4, activation=relu))
network.append(Layer_Dense(4, 1, activation=sigmoid))

# Train netwerk
train(X, y, network, learning_rate=0.3, n_iterations=100000)

# Test netwerk
accuracy = test(X, y, network)
print("Accuracy: ", accuracy)

Accuracy:  0.5
